<a href="https://colab.research.google.com/github/symphonynet/SymphonyNet/blob/main/play_symphonynet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Co-creating music with SymphonyNet

## Install the requirements

In [ ]:
# !git clone https://github.com/symphonynet/SymphonyNet.git

In [ ]:
# !cat SymphonyNet/requirements.txt | xargs -n 1 -L 1 pip install

## Model preparation

In [1]:
MAX_POS_LEN = 4096
PI_LEVEL = 2
IGNORE_META_LOSS = 1
RATIO = 4
BPE = "_bpe" # or ""

DATA_BIN=f"linear_{MAX_POS_LEN}_chord{BPE}_hardloss{IGNORE_META_LOSS}"
CHECKPOINT_SUFFIX=f"{DATA_BIN}_PI{PI_LEVEL}"
DATA_BIN_DIR=f"/home/tnguy231/VIVY/VIVYNet/Decoder/symphony_net/data/model_spec/{DATA_BIN}/bin/"
DATA_VOC_DIR=f"/home/tnguy231/VIVY/VIVYNet/Decoder/symphony_net/data/model_spec/{DATA_BIN}/vocabs/"

from src.fairseq.gen_utils import process_prime_midi, gen_one, get_trk_ins_map, get_note_seq, note_seq_to_midi_file, music_dict
music_dict.load_vocabs_bpe(DATA_VOC_DIR, '/home/tnguy231/VIVY/VIVYNet/Decoder/symphony_net/data/bpe_res/' if BPE == '_bpe' else None)

midi_name = '/home/tnguy231/VIVY/VIVYNet/Decoder/symphony_net/data/midis/ty_maerz_format0.mid'
max_measure_cnt = 5
max_chord_measure_cnt = 0
# prime, ins_label = process_prime_midi(midi_name, max_measure_cnt, max_chord_measure_cnt)
prime1, ins_label1 = process_prime_midi(midi_name, max_measure_cnt, max_chord_measure_cnt)
len(ins_label1)

/home/blherre4/miniconda3/envs/vivyenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '/home/tnguy231/VIVY/VIVYNet/Decoder/symphony_net/data/model_spec/linear_4096_chord_bpe_hardloss1/vocabs/vocab_0.json'

Initialize the model and load pretrained parameters. (You should first save the provided [ckpt file](https://drive.google.com/file/d/1xpkj_qN4MdLRkBdCXmfGjuWWjnTN1Og0/view) into your google drive.)

In [ ]:
from fairseq.models import FairseqLanguageModel
from google.colab import drive
drive.mount('/content/drive')
custom_lm = FairseqLanguageModel.from_pretrained('.', 
    checkpoint_file=f'drive/MyDrive/checkpoint_last_{CHECKPOINT_SUFFIX}.pt', 
    data_name_or_path=DATA_BIN_DIR, 
    user_dir="SymphonyNet/src/fairseq/linear_transformer_inference")

In [ ]:
m = custom_lm.models[0]
m.cuda()
m.eval()

## Prepare prime MIDI

In [ ]:
midi_name = '/home/tnguy231/VIVY/VIVYNet/Decoder/symphony_net/test.mid'
test_midi_name = '/home/tnguy231/VIVY/VIVYNet/Decoder/symphony_net/mz_545_3_format0.mid'
max_measure_cnt = 5
max_chord_measure_cnt = 0
# prime, ins_label = process_prime_midi(midi_name, max_measure_cnt, max_chord_measure_cnt)
prime1, ins_label1 = process_prime_midi(test_midi_name, max_measure_cnt, max_chord_measure_cnt)

In [ ]:
prime[:5]

In [ ]:
ins_label[:5]

## Generation

In [ ]:
import time
while(True):
  try:
    generated, ins_logits = gen_one(m, prime, MIN_LEN = 1024)
    break
  except Exception as e:
    print(e)
    continue
trk_ins_map = get_trk_ins_map(generated, ins_logits)
note_seq = get_note_seq(generated, trk_ins_map)
timestamp = time.strftime("%m-%d_%H-%M-%S", time.localtime()) 
output_name = f'output_prime{max_measure_cnt}_chord{max_chord_measure_cnt}_{timestamp}.mid'
note_seq_to_midi_file(note_seq, output_name)

## Audio Display

In [ ]:
!sudo apt install -y fluidsynth

In [ ]:
!pip install --upgrade pyfluidsynth

In [ ]:
!pip install pretty_midi

In [ ]:
import fluidsynth
import pretty_midi
from IPython import display
_SAMPLING_RATE = 16000
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=300):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [ ]:
pm = pretty_midi.PrettyMIDI(output_name)
display_audio(pm)